In [52]:
using ForwardDiff
using Flux
using Plots; gr()

Plots.GRBackend()

In [47]:
module Att

using Flux
using ForwardDiff

struct Attention{T1, T2}
    signals::T1
    weights::T2
end

function (a::Attention)(x)
    sum(a.weights(x) .* a.signals(x), 1)
end

Flux.params(a::Attention) = vcat(params(a.signals), params(a.weights))

struct TangentPropagator{F <: Function, C}
    f::F
    layer::C
end

function TangentPropagator(chain::Chain)
    f = reduce(∘, identity, _propagate_tangent.(reverse(chain.layers)))
    TangentPropagator(x -> f((x, eye(length(x)))), chain)
end

(p::TangentPropagator)(x) = p.f(x)

Flux.params(p::TangentPropagator) = Flux.params(p.layer)

function _propagate_tangent(f)
    (xJ) -> begin
        (f(xJ[1]), ForwardDiff.jacobian(f, Flux.Tracker.value(xJ[1])) * xJ[2])
    end
end

function _propagate_tangent(f::Dense)
    xJ -> begin
        x, J = xJ
        y = f.W * x + f.b
        gσ = ForwardDiff.derivative.(f.σ, y)
        (f(x), gσ .* f.W * J)
    end
end

function TangentPropagator(a::Attention)
    t1 = TangentPropagator(a.signals)
    t2 = TangentPropagator(a.weights)
    function f(x)
        x1, J1 = t1(x)
        x2, J2 = t2(x)
        sum(x1 .* x2), sum(x1 .* J2, 1) .+ sum(x2 .* J1, 1)
    end
    TangentPropagator(f, a)
end

TangentPropagator(d::Dense) = TangentPropagator(Chain(d))
        

# tangent(f, x) = ForwardDiff.jacobian(f, x)

# function tangent(f::Dense, x)
#     x, J = xJ
#     y = f.W * x + f.b
#     gσ = ForwardDiff.derivative.(f.σ, y)
#     gσ .* f.W * J
# end

end

Att

In [48]:
signals = Dense(1, 2)
weights = Chain(Dense(1, 4, elu), Dense(4, 2, elu), softmax)
model = Att.Attention(signals, weights)

t = Att.TangentPropagator(model)

Att.TangentPropagator{Att.#f#7{Att.TangentPropagator{Att.##1#2{Base.##55#56{Base.#identity,Att.##5#6{Flux.Dense{Base.#identity,TrackedArray{…,Array{Float64,2}},TrackedArray{…,Array{Float64,1}}}}}},Flux.Chain},Att.TangentPropagator{Att.##1#2{Base.##55#56{Base.##55#56{Base.##55#56{Base.#identity,Att.##3#4{NNlib.#softmax}},Att.##5#6{Flux.Dense{NNlib.#elu,TrackedArray{…,Array{Float64,2}},TrackedArray{…,Array{Float64,1}}}}},Att.##5#6{Flux.Dense{NNlib.#elu,TrackedArray{…,Array{Float64,2}},TrackedArray{…,Array{Float64,1}}}}}},Flux.Chain}},Att.Attention{Flux.Dense{Base.#identity,TrackedArray{…,Array{Float64,2}},TrackedArray{…,Array{Float64,1}}},Flux.Chain}}(Att.f, Att.Attention{Flux.Dense{Base.#identity,TrackedArray{…,Array{Float64,2}},TrackedArray{…,Array{Float64,1}}},Flux.Chain}(Dense(1, 2), Chain(Dense(1, 4, NNlib.elu), Dense(4, 2, NNlib.elu), NNlib.softmax)))

In [56]:
eps = 1e-6
x = [1.5]
model(x), (model(x + eps) - model(x)) / eps

(param([-1.23506]), param([-0.835811]))

In [57]:
t(x)

(param(-1.23506), param([-0.835811]))

In [88]:
function oracle(x)
    if x < 2
        -0.5x + 3
    else
        2x - 7
    end
end

xx = linspace(0, 5, 100)
plot(xx, oracle.(xx))

train_x = linspace(0, 5, 5)
train_y = oracle.(train_x)

wrap(x) = [x]
unwrap(x) = (@assert length(x) == 1; Flux.Tracker.value(first(x)))

train_data = [(wrap.(train_x), wrap.(train_y), fill.(ForwardDiff.derivative.(oracle, train_x), 1, 1))]

1-element Array{Tuple{Array{Array{Float64,1},1},Array{Array{Float64,1},1},Array{Array{Float64,2},1}},1}:
 (Array{Float64,1}[[0.0], [1.25], [2.5], [3.75], [5.0]], Array{Float64,1}[[3.0], [2.375], [-2.0], [0.5], [3.0]], Array{Float64,2}[[-0.5], [-0.5], [2.0], [2.0], [2.0]])

In [107]:
signals = Dense(1, 2)
weights = Chain(Dense(1, 4, elu), Dense(4, 2, elu), softmax)
model = Att.Attention(signals, weights)
tmodel = Att.TangentPropagator(model)

λ = 0.05

loss = (x, y, J) -> begin
    yJ = collect(zip(y, J))
    yJhat = tmodel.(x)
    
    function sample_error(yJi, yJhati)
        yi, Ji = yJi
        yhati, Jhati = yJhati
        (1 - λ) * sum(abs2.(yi .- yhati)) + λ * sum(abs2.(Ji .- Jhati))
    end
    sum(sample_error.(yJ, yJhat)) / length(x)
end
opt = Flux.Optimise.ADADelta(params(tmodel))

(::#71) (generic function with 1 method)

In [108]:
xx = linspace(0, 5, 100)
plt = plot(xx, oracle.(xx))
plot!(plt, xx, unwrap.(model.(wrap.(xx))))
scatter!(plt, train_x, train_y)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
<polyline clip-path="url(#clip8202)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.8551,22.113 50.1062,23.5681 55.3574,25.0231 60.6085,26.4782 65.8597,27.9333 71.1108,29.3884 76.362,30.8434 81.6131,32.2985 86.8643,33.7536 92.1154,35.2087 
 97.3666,36.6638 102.618,38.1188 107.869,39.5739 113.12,41.029 118.371,42.4841 123.622,43.9392 128.873,45.3942 134.125,46.8493 139.376,48.3044 144.627,49.7595 
 149.878,51.2145 155.129,52.6696 160.38,54.1247 165.632,55.5798 170.883,57.0349 176.134,58.4899 181.385,59.945 186.636,61.4001 191.887,62.8552 197.138,64.3103 
 202.39,65.7653 207.641,67.2204 212.892,68.6755 218.143,70.1306 223.394,71.5856 228.645,73.0407 233.897,74.4958 239.148,75.9509 244.399,77.406 249.65,78.861 
 254.901,365.511 260.152,359.691 265.403,353.871 270.655,348.051 275.906,342.23 281.157,336.41 286.408,330.59 291.659,324.769 296.91,318.949 302.161,313.129 
 307.413,307.308 312.664,301.488 317.915,295.668 323.166,289.847 328.417,284.027 333.668,278.207 338.92,272.386 344.171,266.566 349.422,260.746 354.673,254.926 
 359.924,249.105 365.175,243.285 370.426,237.465 375.678,231.644 380.929,225.824 386.18,220.004 391.431,214.183 396.682,208.363 401.933,202.543 407.185,196.722 
 412.436,190.902 417.687,185.082 422.938,179.261 428.189,173.441 433.44,167.621 438.691,161.801 443.943,155.98 449.194,150.16 454.445,144.34 459.696,138.519 
 464.947,132.699 470.198,126.879 475.449,121.058 480.701,115.238 485.952,109.418 491.203,103.597 496.454,97.7771 501.705,91.9567 506.956,86.1364 512.208,80.3161 
 517.459,74.4958 522.71,68.6755 527.961,62.8552 533.212,57.0349 538.463,51.2145 543.714,45.3942 548.966,39.5739 554.217,33.7536 559.468,27.9333 564.719,22.113 
 
 "/>
<polyline clip-path="url(#clip8202)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.8551,194.976 50.1062,193.813 55.3574,192.641 60.6085,191.46 65.8597,190.27 71.1108,189.072 76.362,187.866 81.6131,186.652 86.8643,185.431 92.1154,184.202 
 97.3666,182.967 102.618,181.724 107.869,180.476 113.12,179.221 118.371,177.96 123.622,176.694 128.873,175.422 134.125,174.144 139.376,172.862 144.627,171.575 
 149.878,170.283 155.129,168.987 160.38,167.686 165.632,166.382 170.883,165.073 176.134,163.761 181.385,162.445 186.636,161.125 191.887,159.802 197.138,158.476 
 202.39,157.147 207.641,155.815 212.892,154.479 218.143,153.141 223.394,151.8 228.645,150.457 233.897,149.111 239.148,147.762 244.399,146.411 249.65,145.058 
 254.901,143.702 260.152,142.344 265.403,140.984 270.655,139.622 275.906,138.257 281.157,136.891 286.408,135.522 291.659,134.152 296.91,132.78 302.161,131.406 
 307.413,130.03 312.664,128.652 317.915,127.272 323.166,125.891 328.417,124.508 333.668,123.123 338.92,121.736 344.171,120.348 349.422,118.958 354.673,117.567 
 359.924,116.174 365.175,114.779 370.426,113.383 375.678,111.985 380.929,110.586 386.18,109.185 391.431,107.783 396.682,106.379 401.933,104.974 407.185,103.567 
 412.436,102.159 417.687,100.75 422.938,99.3391 428.189,97.927 433.44,96.5135 438.691,95.0988 443.943,93.6827 449.194,92.2653 454.445,90.8467 459.696,89.4268 
 464.947,88.0057 470.198,86.5833 475.449,85.1598 480.701,83.7351 485.952,82.3092 491.203,80.8821 496.454,79.454 501.705,78.0247 506.956,76.5943 512.208,75.1629 
 517.459,73.7304 522.71,72.2968 527.961,70.8623 533.212,69.4267 538.463,67.9902 543.714,66.5526 548.966,65.1142 554.217,63.6748 559.468,62.2345 564.719,60.7933 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 y2 
 
 
 
 
 y3

In [103]:
for i in 1:5000
    Flux.train!(loss, train_data, opt)
end

In [104]:
xx = linspace(0, 5, 100)
plt = plot(xx, oracle.(xx))
plot!(plt, xx, unwrap.(model.(wrap.(xx))))
scatter!(plt, train_x, train_y)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
<polyline clip-path="url(#clip7802)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.8551,22.3626 50.1062,23.8166 55.3574,25.2706 60.6085,26.7246 65.8597,28.1786 71.1108,29.6327 76.362,31.0867 81.6131,32.5407 86.8643,33.9947 92.1154,35.4487 
 97.3666,36.9028 102.618,38.3568 107.869,39.8108 113.12,41.2648 118.371,42.7189 123.622,44.1729 128.873,45.6269 134.125,47.0809 139.376,48.5349 144.627,49.989 
 149.878,51.443 155.129,52.897 160.38,54.351 165.632,55.805 170.883,57.2591 176.134,58.7131 181.385,60.1671 186.636,61.6211 191.887,63.0751 197.138,64.5292 
 202.39,65.9832 207.641,67.4372 212.892,68.8912 218.143,70.3452 223.394,71.7993 228.645,73.2533 233.897,74.7073 239.148,76.1613 244.399,77.6154 249.65,79.0694 
 254.901,365.511 260.152,359.695 265.403,353.879 270.655,348.063 275.906,342.247 281.157,336.431 286.408,330.615 291.659,324.799 296.91,318.983 302.161,313.167 
 307.413,307.351 312.664,301.535 317.915,295.718 323.166,289.902 328.417,284.086 333.668,278.27 338.92,272.454 344.171,266.638 349.422,260.822 354.673,255.006 
 359.924,249.19 365.175,243.374 370.426,237.558 375.678,231.742 380.929,225.925 386.18,220.109 391.431,214.293 396.682,208.477 401.933,202.661 407.185,196.845 
 412.436,191.029 417.687,185.213 422.938,179.397 428.189,173.581 433.44,167.765 438.691,161.949 443.943,156.132 449.194,150.316 454.445,144.5 459.696,138.684 
 464.947,132.868 470.198,127.052 475.449,121.236 480.701,115.42 485.952,109.604 491.203,103.788 496.454,97.9716 501.705,92.1556 506.956,86.3395 512.208,80.5234 
 517.459,74.7073 522.71,68.8912 527.961,63.0751 533.212,57.2591 538.463,51.443 543.714,45.6269 548.966,39.8108 554.217,33.9947 559.468,28.1786 564.719,22.3626 
 
 "/>
<polyline clip-path="url(#clip7802)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.8551,22.113 50.1062,23.5425 55.3574,25 60.6085,26.48 65.8597,27.9769 71.1108,29.485 76.362,30.9982 81.6131,32.5107 86.8643,34.0166 92.1154,35.5102 
 97.3666,36.9865 102.618,38.4407 107.869,39.8691 113.12,41.269 118.371,42.6388 123.622,43.9787 128.873,45.2906 134.125,46.5787 139.376,47.8497 144.627,49.1135 
 149.878,50.3832 155.129,51.6763 160.38,53.0149 165.632,54.4263 170.883,55.9444 176.134,57.6461 181.385,60.2307 186.636,64.0994 191.887,69.4847 197.138,76.6755 
 202.39,86.0223 207.641,97.9334 212.892,112.853 218.143,131.204 223.394,153.274 228.645,179.024 233.897,207.843 239.148,236.153 244.399,260.604 249.65,280.444 
 254.901,295.685 260.152,306.711 265.403,314.094 270.655,318.462 275.906,320.413 281.157,320.47 286.408,319.068 291.659,316.555 296.91,313.203 302.161,309.221 
 307.413,304.768 312.664,299.963 317.915,294.895 323.166,289.631 328.417,284.222 333.668,278.704 338.92,273.104 344.171,267.444 349.422,261.739 354.673,255.999 
 359.924,250.235 365.175,244.45 370.426,238.65 375.678,232.839 380.929,227.02 386.18,221.196 391.431,215.367 396.682,209.535 401.933,203.701 407.185,197.865 
 412.436,192.027 417.687,186.189 422.938,180.349 428.189,174.509 433.44,168.669 438.691,162.828 443.943,156.986 449.194,151.145 454.445,145.303 459.696,139.461 
 464.947,133.619 470.198,127.776 475.449,121.934 480.701,116.092 485.952,110.249 491.203,104.407 496.454,98.5642 501.705,92.7217 506.956,86.8791 512.208,81.0365 
 517.459,75.1938 522.71,69.3512 527.961,63.5085 533.212,57.6659 538.463,51.8232 543.714,45.9805 548.966,40.1378 554.217,34.2951 559.468,28.4524 564.719,22.6097 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 y2 
 
 
 
 
 y3

In [113]:
signals = Dense(1, 2)
weights = Chain(Dense(1, 4, elu), Dense(4, 2, elu), softmax)
model = Att.Attention(signals, weights)

loss = (x, y, J) -> sum(z -> sum(abs2.(z)), model.(x) .- y) / length(x)
opt = Flux.Optimise.ADADelta(params(model))

(::#71) (generic function with 1 method)

In [114]:
xx = linspace(0, 5, 100)
plt = plot(xx, oracle.(xx))
plot!(plt, xx, unwrap.(model.(wrap.(xx))))
scatter!(plt, train_x, train_y)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
<polyline clip-path="url(#clip8602)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.8551,22.113 50.1062,23.2819 55.3574,24.4508 60.6085,25.6198 65.8597,26.7887 71.1108,27.9577 76.362,29.1266 81.6131,30.2955 86.8643,31.4645 92.1154,32.6334 
 97.3666,33.8023 102.618,34.9713 107.869,36.1402 113.12,37.3091 118.371,38.4781 123.622,39.647 128.873,40.8159 134.125,41.9849 139.376,43.1538 144.627,44.3228 
 149.878,45.4917 155.129,46.6606 160.38,47.8296 165.632,48.9985 170.883,50.1674 176.134,51.3364 181.385,52.5053 186.636,53.6742 191.887,54.8432 197.138,56.0121 
 202.39,57.181 207.641,58.35 212.892,59.5189 218.143,60.6878 223.394,61.8568 228.645,63.0257 233.897,64.1947 239.148,65.3636 244.399,66.5325 249.65,67.7015 
 254.901,297.982 260.152,293.306 265.403,288.63 270.655,283.955 275.906,279.279 281.157,274.603 286.408,269.927 291.659,265.252 296.91,260.576 302.161,255.9 
 307.413,251.224 312.664,246.549 317.915,241.873 323.166,237.197 328.417,232.521 333.668,227.846 338.92,223.17 344.171,218.494 349.422,213.818 354.673,209.143 
 359.924,204.467 365.175,199.791 370.426,195.115 375.678,190.44 380.929,185.764 386.18,181.088 391.431,176.412 396.682,171.737 401.933,167.061 407.185,162.385 
 412.436,157.709 417.687,153.034 422.938,148.358 428.189,143.682 433.44,139.007 438.691,134.331 443.943,129.655 449.194,124.979 454.445,120.304 459.696,115.628 
 464.947,110.952 470.198,106.276 475.449,101.601 480.701,96.9248 485.952,92.2491 491.203,87.5734 496.454,82.8976 501.705,78.2219 506.956,73.5461 512.208,68.8704 
 517.459,64.1947 522.71,59.5189 527.961,54.8432 533.212,50.1674 538.463,45.4917 543.714,40.8159 548.966,36.1402 554.217,31.4645 559.468,26.7887 564.719,22.113 
 
 "/>
<polyline clip-path="url(#clip8602)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.8551,160.983 50.1062,161.719 55.3574,162.541 60.6085,163.444 65.8597,164.424 71.1108,165.475 76.362,166.593 81.6131,167.775 86.8643,169.016 92.1154,170.314 
 97.3666,171.664 102.618,173.065 107.869,174.513 113.12,176.005 118.371,177.541 123.622,179.116 128.873,180.729 134.125,182.378 139.376,184.062 144.627,185.779 
 149.878,187.526 155.129,189.304 160.38,191.109 165.632,192.942 170.883,194.8 176.134,196.683 181.385,198.59 186.636,200.519 191.887,202.47 197.138,204.441 
 202.39,206.433 207.641,208.443 212.892,210.472 218.143,212.518 223.394,214.581 228.645,216.66 233.897,218.755 239.148,220.864 244.399,222.988 249.65,225.125 
 254.901,227.276 260.152,229.44 265.403,231.615 270.655,233.802 275.906,236.001 281.157,238.21 286.408,240.43 291.659,242.66 296.91,244.899 302.161,247.148 
 307.413,249.405 312.664,251.671 317.915,253.945 323.166,256.226 328.417,258.515 333.668,260.812 338.92,263.115 344.171,265.424 349.422,267.74 354.673,270.062 
 359.924,272.39 365.175,274.723 370.426,277.061 375.678,279.404 380.929,281.752 386.18,284.104 391.431,286.461 396.682,288.822 401.933,291.186 407.185,293.554 
 412.436,295.926 417.687,298.301 422.938,300.679 428.189,303.06 433.44,305.443 438.691,307.829 443.943,310.218 449.194,312.609 454.445,315.001 459.696,317.396 
 464.947,319.793 470.198,322.191 475.449,324.591 480.701,326.992 485.952,329.394 491.203,331.798 496.454,334.203 501.705,336.608 506.956,339.015 512.208,341.422 
 517.459,343.829 522.71,346.238 527.961,348.646 533.212,351.055 538.463,353.464 543.714,355.874 548.966,358.283 554.217,360.693 559.468,363.102 564.719,365.511 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 y2 
 
 
 
 
 y3

In [121]:
for i in 1:5000
    Flux.train!(loss, train_data, opt)
end

In [122]:
xx = linspace(0, 5, 100)
plt = plot(xx, oracle.(xx))
plot!(plt, xx, unwrap.(model.(wrap.(xx))))
scatter!(plt, train_x, train_y)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
<polyline clip-path="url(#clip9402)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.8551,25.6367 50.1062,27.0768 55.3574,28.517 60.6085,29.9571 65.8597,31.3973 71.1108,32.8374 76.362,34.2776 81.6131,35.7177 86.8643,37.1579 92.1154,38.598 
 97.3666,40.0381 102.618,41.4783 107.869,42.9184 113.12,44.3586 118.371,45.7987 123.622,47.2389 128.873,48.679 134.125,50.1192 139.376,51.5593 144.627,52.9995 
 149.878,54.4396 155.129,55.8798 160.38,57.3199 165.632,58.7601 170.883,60.2002 176.134,61.6404 181.385,63.0805 186.636,64.5207 191.887,65.9608 197.138,67.401 
 202.39,68.8411 207.641,70.2812 212.892,71.7214 218.143,73.1615 223.394,74.6017 228.645,76.0418 233.897,77.482 239.148,78.9221 244.399,80.3623 249.65,81.8024 
 254.901,365.511 260.152,359.751 265.403,353.99 270.655,348.23 275.906,342.469 281.157,336.709 286.408,330.948 291.659,325.187 296.91,319.427 302.161,313.666 
 307.413,307.906 312.664,302.145 317.915,296.384 323.166,290.624 328.417,284.863 333.668,279.103 338.92,273.342 344.171,267.581 349.422,261.821 354.673,256.06 
 359.924,250.3 365.175,244.539 370.426,238.779 375.678,233.018 380.929,227.257 386.18,221.497 391.431,215.736 396.682,209.976 401.933,204.215 407.185,198.454 
 412.436,192.694 417.687,186.933 422.938,181.173 428.189,175.412 433.44,169.651 438.691,163.891 443.943,158.13 449.194,152.37 454.445,146.609 459.696,140.848 
 464.947,135.088 470.198,129.327 475.449,123.567 480.701,117.806 485.952,112.046 491.203,106.285 496.454,100.524 501.705,94.7638 506.956,89.0032 512.208,83.2426 
 517.459,77.482 522.71,71.7214 527.961,65.9608 533.212,60.2002 538.463,54.4396 543.714,48.679 548.966,42.9184 554.217,37.1579 559.468,31.3973 564.719,25.6367 
 
 "/>
<polyline clip-path="url(#clip9402)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.8551,25.6605 50.1062,25.3401 55.3574,25.029 60.6085,24.7294 65.8597,24.444 71.1108,24.1761 76.362,23.9298 81.6131,23.7101 86.8643,23.5235 92.1154,23.3778 
 97.3666,23.2829 102.618,23.2513 107.869,23.2986 113.12,23.445 118.371,23.7158 123.622,24.144 128.873,24.7721 134.125,25.6624 139.376,26.9124 144.627,28.6444 
 149.878,31.0273 155.129,34.2967 160.38,38.7807 165.632,44.9025 170.883,52.8484 176.134,62.8839 181.385,75.3004 186.636,90.3057 191.887,107.96 197.138,128.117 
 202.39,150.387 207.641,173.573 212.892,194.6 218.143,212.968 223.394,228.778 228.645,242.287 233.897,253.802 239.148,263.616 244.399,271.986 249.65,279.124 
 254.901,285.203 260.152,290.355 265.403,294.685 270.655,298.271 275.906,301.173 281.157,303.435 286.408,305.09 291.659,306.161 296.91,306.664 302.161,306.609 
 307.413,306.005 312.664,304.857 317.915,303.168 323.166,300.942 328.417,298.183 333.668,294.897 338.92,291.091 344.171,286.775 349.422,281.961 354.673,276.666 
 359.924,270.91 365.175,264.714 370.426,258.106 375.678,251.115 380.929,243.775 386.18,236.123 391.431,228.214 396.682,220.094 401.933,211.806 407.185,203.393 
 412.436,194.902 417.687,186.372 422.938,177.845 428.189,169.36 433.44,160.953 438.691,152.656 443.943,144.501 449.194,136.515 454.445,128.722 459.696,121.141 
 464.947,113.791 470.198,106.686 475.449,99.8345 480.701,93.2461 485.952,86.9253 491.203,80.8748 496.454,75.0947 501.705,69.5835 506.956,64.3377 512.208,59.3525 
 517.459,54.6216 522.71,50.138 527.961,45.8936 533.212,41.8797 538.463,38.0874 543.714,34.507 548.966,31.1291 554.217,27.9439 559.468,24.9417 564.719,22.113 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 y2 
 
 
 
 
 y3